In [1]:
import zarr
import allel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from ag3 import release_data

/opt/conda/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
v3 = release_data()

In [4]:
import fsspec

import gcsfs

In [5]:
gcsfs.core.client_config['installed']['client_id'] = '160816776314-7h1p2h6bqhe77vs1su4rr22fblso9dre.apps.googleusercontent.com'
gcsfs.core.client_config['installed']['client_secret'] = 'SrJUwD58yKgViyJtw-NP2sz8'

In [6]:
# create a GCSFileSystem just for the purpose of authentication
gcs_orig = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='cache', cache_timeout=0)

# create another one with those credentials
gcs =  gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token=gcs_orig.session.credentials, cache_timeout=0)

In [7]:
aim_path = 'gs://vo_agam_production/aim_results/{sample_set}.zarr'

## Overview

- Step 1 given large number of AIMs, lets be more discerning and filter them.

What are the frequencies of each allele in gamb_colu, and arabiensis?

In [8]:
import dask.array as da

In [63]:
chromosomes = "2R", "2L", "3R", "3L", "X"

In [57]:
# loop through all samplesets and chromosomes
keep_aim = {}
for chrom in chromosomes:
    ac_arabiensis = []
    ac_gambiae = []
    for sample_set in v3.all_wild_sample_sets:

        aim_store = fsspec.get_mapper(aim_path.format(sample_set=sample_set))
        aim_data = zarr.open_consolidated(aim_store)

        metadata = v3.load_sample_set_metadata(sample_set)

        subpops = metadata.groupby(["species_gambcolu_arabiensis"]).indices

        g = allel.GenotypeArray(da.from_zarr(aim_data["gambcolu_vs_arab"][chrom]["GT_AIM"]).compute())

        ac = g.count_alleles_subpops(subpops)

        try:
            ac_arabiensis.append(ac["arabiensis"])
        except KeyError:
            pass

        try:
            ac_gambiae.append(ac["gamb_colu"])
        except KeyError:
            pass

    ac_gambcolu = allel.AlleleCountsArray(np.stack(ac_gambiae, axis=2).sum(axis=2))
    alt_freq_gamb_colu = ac_gambcolu.to_frequencies()[:, 1]

    ac_arab = allel.AlleleCountsArray(np.stack(ac_arabiensis, axis=2).sum(axis=2))
    alt_freq_arab = ac_arab.to_frequencies()[:, 1]

    if chrom == "X":
        keep_aim[chrom] = (alt_freq_arab >= 0.8) & (np.random.random(alt_freq_arab.shape[0]) > 0.9)
    else:
        keep_aim[chrom] = alt_freq_arab >= 0.8

/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning:

In [58]:
aims_per_chrom = [keep_aim[chrom].sum() for chrom in chromosomes]
aims_per_chrom

[1658, 1439, 2176, 1981, 2439]

In [59]:
sample_set_sizes = [v3.load_sample_set_calldata("X", sample_set).shape[1] for sample_set in v3.all_wild_sample_sets]

In [60]:
from matplotlib.gridspec import GridSpec

In [ ]:
# now compress the AIMs by the keep list, and plot.

# loop through all samplesets.
fig = plt.figure(figsize=(20, 15))

gs = GridSpec(nrows=len(v3.all_wild_sample_sets), ncols=len(chromosomes), height_ratios=sample_set_sizes, width_ratios=aims_per_chrom)

for i, sample_set in enumerate(v3.all_wild_sample_sets):
    
    aim_store = fsspec.get_mapper(aim_path.format(sample_set=sample_set))
    aim_data = zarr.open_consolidated(aim_store)

    metadata = v3.load_sample_set_metadata(sample_set)

    subpops = metadata.groupby(["species_gambcolu_arabiensis"]).indices
    
    for j, chrom in enumerate(chromosomes):

        g = allel.GenotypeArray(da.from_zarr(aim_data["gambcolu_vs_arab"][chrom]["GT_AIM"]).compute()).compress(keep_aim[chrom], axis=0)

        nalt = g.to_n_alt()
        # sort by n_arab alleles.

        count_arab_alleles = nalt.sum(axis=0)

        ix = np.argsort(count_arab_alleles)
        nalt_sorted = np.take(nalt, ix, axis=1)

        ax = fig.add_subplot(gs[i, j])
        ax.pcolormesh(nalt_sorted.T, cmap="Dark2")
        ax.set_yticks([])
        ax.set_xticks([])
        if chrom == "2R":
            ax.set_ylabel(sample_set, rotation=0, horizontalalignment="right", verticalalignment="center")
            
fig.savefig("../content/images/aim/gambcolu_vs_arabiensis_aim.png", dpi=200)

/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
